# RNN with LSTM with Own Dataset

In [1]:
import torch
import torchtext
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
import spacy
nlp = spacy.load("en")

In this lab one will use own CSV text dataset for training a simple RNN for sentiment classification (here: a binary classification problem with two labels, positive and negative) using LSTM (Long Short Term Memory) cells and GRU Cells.

In [3]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random
import pandas as pd

torch.backends.cudnn.deterministic = True

## General Settings

In [4]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

## Dataset

The following cells will download the IMDB movie review dataset (http://ai.stanford.edu/~amaas/data/sentiment/) for positive-negative sentiment classification in as CSV-formatted file:

In [5]:
!gunzip -f movie_data.csv.gz 

gunzip: can't stat: movie_data.csv.gz (movie_data.csv.gz.gz): No such file or directory


In [6]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


Check that the dataset looks okay:

In [7]:
del df

Define the Label and Text field formatters:

In [8]:
TEXT = data.Field(sequential=True,
                  tokenize='spacy',
                  include_lengths=True) # necessary for packed_padded_sequence

LABEL = data.LabelField(dtype=torch.float)

/Users/lilytang/opt/anaconda3/lib/python3.9/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/lilytang/opt/anaconda3/lib/python3.9/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Process the dataset:

In [9]:
fields = [('review', TEXT), ('sentiment', LABEL)]

dataset = data.TabularDataset(
    path="movie_data.csv", format='csv',
    skip_header=True, fields=fields)

/Users/lilytang/opt/anaconda3/lib/python3.9/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/Users/lilytang/opt/anaconda3/lib/python3.9/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Split the dataset into training, validation, and test partitions:

In [10]:
train_data, valid_data, test_data = dataset.split(
    split_ratio=[0.75, 0.05, 0.2],
    random_state=random.seed(RANDOM_SEED))
#One may want to vary the test, train split percentages
print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 37500
Num Valid: 10000
Num Test: 2500


Build the vocabulary based on the top "VOCABULARY_SIZE" words:

In [11]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [12]:
LABEL.vocab.freqs

Counter({'0': 18742, '1': 18758})

The TEXT.vocab dictionary will contain the word counts and indices. The reason why the number of words is VOCABULARY_SIZE + 2 is that it contains to special tokens for padding and unknown words: `<unk>` and `<pad>`.

Make dataset iterators:

In [13]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    sort_key=lambda x: len(x.review),
    device=DEVICE)

/Users/lilytang/opt/anaconda3/lib/python3.9/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

In [14]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break

Train
Text matrix size: torch.Size([512, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([52, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([74, 128])
Target vector size: torch.Size([128])


/Users/lilytang/opt/anaconda3/lib/python3.9/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Model

 ### The primary goal of this lab is to vary the hyperparameters of the LSTM model and see the results and provide analysis
 ### The second task is to use a another RNN cell such as GRU and perform parameter tuning and report the results.
 
 ### The remainder of the code will have to be modified accordingly. 

## 1. Originally model

In [30]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        #Here is a preliminary model using LSTM cell
        #The primary goal of this lab is to vary the dimensions of the embeddings and see the results
        #The second task is to use a another RNN cell such as GRU and perform parameter tuning and report the results.
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.rnn(packed)
        
        return self.fc(hidden.squeeze(0)).view(-1)

In [31]:
INPUT_DIM = len(TEXT.vocab)

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### Training

In [32]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.review
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.sentiment.size(0)
            correct_pred += (predicted_labels.long() == batch_data.sentiment.long()).sum()
        return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.review
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.sentiment)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
print('Probability positive:')
1-predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability negative:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

## 2. Task1: Vary the hyperparameters of the LSTM model

In [19]:
import torch.nn as nn 

EMBEDDING_DIM = 256 ## increased from 128 to 256
HIDDEN_DIM = 512  ## increased from 256 to 512
NUM_LAYERS = 2   ## add new number of layers parameter
BIDIRECTIONAL = True ## add new parameter
DROPOUT = 0.5  ## add new dropout parameter


class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout):
        
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
    def forward(self, text, text_length):
        embedded = self.embedding(text)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        packed_output, (hidden, cell) = self.rnn(packed)
        if self.rnn.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden.squeeze(0)).view(-1)

INPUT_DIM = len(TEXT.vocab)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT)
torch.manual_seed(RANDOM_SEED)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### Training

In [20]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.review
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.sentiment.size(0)
            correct_pred += (predicted_labels.long() == batch_data.sentiment.long()).sum()
        return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.review
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.sentiment)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/293 | Cost: 0.6934


In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
print('Probability positive:')
1-predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability negative:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

## 2. Task2: use a another RNN cell such as GRU and perform parameter tuning 

In [15]:
import torch.nn as nn 

#EMBEDDING_DIM = 256 ## increased from 128 to 256
#HIDDEN_DIM = 512  ## increased from 256 to 512
#NUM_LAYERS = 2   ## add new number of layers parameter
#BIDIRECTIONAL = True ## add new parameter
#DROPOUT = 0.5  ## add new dropout parameter

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout):
        
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        ## change from lstm to GRU
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
    def forward(self, text, text_length):
        embedded = self.embedding(text)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        packed_output, hidden = self.rnn(packed)
        if self.rnn.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden.squeeze(0)).view(-1)


### Parameter tuning 

In [ ]:
# Define hyperparameters to tune
learning_rates = [0.001, 0.01, 0.1]
hidden_dims = [64, 128, 256]
batch_sizes = [32, 64, 128]

results = {}

for lr in learning_rates:
    for hd in hidden_dims:
        for bs in batch_sizes:
            print(f"Training with lr={lr}, hd={hd}, bs={bs}")
            
            # Define the model with the new hyperparameters
            EMBEDDING_DIM = 256
            HIDDEN_DIM = hd
            NUM_LAYERS = 2
            BIDIRECTIONAL = True
            DROPOUT = 0.5
            INPUT_DIM = len(TEXT.vocab)
            OUTPUT_DIM = 1

            model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT)
            torch.manual_seed(RANDOM_SEED)
            model = model.to(DEVICE)

            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            criterion = nn.BCEWithLogitsLoss().to(DEVICE)

            # Retrain the model
            start_time = time.time()
            for epoch in range(NUM_EPOCHS):
                model.train()
                for batch_idx, batch_data in enumerate(train_loader):
                    text, text_lengths = batch_data.review
                    logits = model(text, text_lengths)
                    cost = criterion(logits, batch_data.sentiment)
                    optimizer.zero_grad()
                    cost.backward()
                    optimizer.step()

            # Compute the test accuracy with the new hyperparameters
            test_acc = compute_binary_accuracy(model, test_loader, DEVICE)

            # Save the results
            results[(lr, hd, bs)] = test_acc

            print(f"Test accuracy: {test_acc:.2f}%")
            print(f"Total Training Time: {(time.time() - start_time)/60:.2f} min")


Training with lr=0.001, hd=64, bs=32


In [ ]:
print('Probability positive:')
1-predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability negative:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")